<a href="https://colab.research.google.com/github/SusanSagwa/Energy-Consumption-Prediction-In-Smart-Buildings-Using-Ensemble-Learning/blob/main/machine_project_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import StackingRegressor
from sklearn.model_selection import train_test_split

In [3]:
DATASET_PATH  = '/content/drive/MyDrive/EPdata'

In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/EPdata/train.csv')


In [5]:
trainData = dataset
trainData

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.000
1,1,0,2016-01-01 00:00:00,0.000
2,2,0,2016-01-01 00:00:00,0.000
3,3,0,2016-01-01 00:00:00,0.000
4,4,0,2016-01-01 00:00:00,0.000
...,...,...,...,...
20216095,1444,0,2016-12-31 23:00:00,8.750
20216096,1445,0,2016-12-31 23:00:00,4.825
20216097,1446,0,2016-12-31 23:00:00,0.000
20216098,1447,0,2016-12-31 23:00:00,159.575


In [6]:
dataset.isnull().sum()

building_id      0
meter            0
timestamp        0
meter_reading    0
dtype: int64

In [7]:
buildingData = pd.read_csv(DATASET_PATH + '/building_metadata.csv')

In [8]:
buildingData.isnull().sum()

site_id           0
building_id       0
primary_use       0
square_feet       0
year_built      774
floor_count    1094
dtype: int64

# Merging building data with training data

In [9]:
data = dataset.merge(buildingData,on = 'building_id',how = 'left')

# Removing features having alot of nulls 

In [10]:
data = data.drop('year_built',axis = 1)
data = data.drop('floor_count',axis = 1)

In [11]:
weather_train = pd.read_csv(DATASET_PATH + '/weather_train.csv')
weather_test = weather_df = pd.read_csv(DATASET_PATH + '/weather_test.csv')

# Concatinating weather train and weather test data

In [12]:
import datetime as dt
frames = [weather_train,weather_test]
weather_data = pd.concat(frames)
data = data.merge(weather_data, on=['site_id', 'timestamp'], how='left')


# Converting timestamp feature to datetime to extract another features such as day,hour,month,and year

In [13]:
data["timestamp"] = pd.to_datetime(data["timestamp"])
data["day"]       = data["timestamp"].dt.day
data["hour"]      = data["timestamp"].dt.hour
data["week"]      = data["timestamp"].dt.weekday
data["month"]     = data["timestamp"].dt.month
data["year"]      = data["timestamp"].dt.year
data              = data.drop("timestamp", axis = 1)
del weather_data

In [14]:
data

,building_id,meter,meter_reading,site_id,primary_use,square_feet,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,hour,week,month,year
0,0,0,0.000,0,Education,7432,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,4,1,2016
1,1,0,0.000,0,Education,2720,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,4,1,2016
2,2,0,0.000,0,Education,5376,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,4,1,2016
3,3,0,0.000,0,Education,23685,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,4,1,2016
4,4,0,0.000,0,Education,116607,25.0,6.0,20.0,NaN,1019.7,0.0,0.0,1,0,4,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,8.750,15,Entertainment/public assembly,19619,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,31,23,5,12,2016
20216096,1445,0,4.825,15,Education,4298,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,31,23,5,12,2016
20216097,1446,0,0.000,15,Entertainment/public assembly,11265,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,31,23,5,12,2016
20216098,1447,0,159.575,15,Lodging/residential,29775,1.7,NaN,-5.6,-1.0,1008.5,180.0,8.8,31,23,5,12,2016


In [15]:
data.isnull().sum()

building_id                 0
meter                       0
meter_reading               0
site_id                     0
primary_use                 0
square_feet                 0
air_temperature         96658
cloud_coverage        8825365
dew_temperature        100140
precip_depth_1_hr     3749023
sea_level_pressure    1231669
wind_direction        1449048
wind_speed             143676
day                         0
hour                        0
week                        0
month                       0
year                        0
dtype: int64

# Removing useless features

In [16]:
data.isnull().sum()

building_id                 0
meter                       0
meter_reading               0
site_id                     0
primary_use                 0
square_feet                 0
air_temperature         96658
cloud_coverage        8825365
dew_temperature        100140
precip_depth_1_hr     3749023
sea_level_pressure    1231669
wind_direction        1449048
wind_speed             143676
day                         0
hour                        0
week                        0
month                       0
year                        0
dtype: int64

In [17]:
# precip_depth_1_hr
data = data.drop('cloud_coverage',axis = 1)
data

,building_id,meter,meter_reading,site_id,primary_use,square_feet,air_temperature,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed,day,hour,week,month,year
0,0,0,0.000,0,Education,7432,25.0,20.0,NaN,1019.7,0.0,0.0,1,0,4,1,2016
1,1,0,0.000,0,Education,2720,25.0,20.0,NaN,1019.7,0.0,0.0,1,0,4,1,2016
2,2,0,0.000,0,Education,5376,25.0,20.0,NaN,1019.7,0.0,0.0,1,0,4,1,2016
3,3,0,0.000,0,Education,23685,25.0,20.0,NaN,1019.7,0.0,0.0,1,0,4,1,2016
4,4,0,0.000,0,Education,116607,25.0,20.0,NaN,1019.7,0.0,0.0,1,0,4,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,8.750,15,Entertainment/public assembly,19619,1.7,-5.6,-1.0,1008.5,180.0,8.8,31,23,5,12,2016
20216096,1445,0,4.825,15,Education,4298,1.7,-5.6,-1.0,1008.5,180.0,8.8,31,23,5,12,2016
20216097,1446,0,0.000,15,Entertainment/public assembly,11265,1.7,-5.6,-1.0,1008.5,180.0,8.8,31,23,5,12,2016
20216098,1447,0,159.575,15,Lodging/residential,29775,1.7,-5.6,-1.0,1008.5,180.0,8.8,31,23,5,12,2016


In [18]:
data = data.drop(['wind_direction','sea_level_pressure'],axis=1)
data

,building_id,meter,meter_reading,site_id,primary_use,square_feet,air_temperature,dew_temperature,precip_depth_1_hr,wind_speed,day,hour,week,month,year
0,0,0,0.000,0,Education,7432,25.0,20.0,NaN,0.0,1,0,4,1,2016
1,1,0,0.000,0,Education,2720,25.0,20.0,NaN,0.0,1,0,4,1,2016
2,2,0,0.000,0,Education,5376,25.0,20.0,NaN,0.0,1,0,4,1,2016
3,3,0,0.000,0,Education,23685,25.0,20.0,NaN,0.0,1,0,4,1,2016
4,4,0,0.000,0,Education,116607,25.0,20.0,NaN,0.0,1,0,4,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,8.750,15,Entertainment/public assembly,19619,1.7,-5.6,-1.0,8.8,31,23,5,12,2016
20216096,1445,0,4.825,15,Education,4298,1.7,-5.6,-1.0,8.8,31,23,5,12,2016
20216097,1446,0,0.000,15,Entertainment/public assembly,11265,1.7,-5.6,-1.0,8.8,31,23,5,12,2016
20216098,1447,0,159.575,15,Lodging/residential,29775,1.7,-5.6,-1.0,8.8,31,23,5,12,2016


# Filling missing data

In [19]:
data['air_temperature'] = data['air_temperature'].fillna(data['air_temperature'].mean())
data['wind_speed'] = data['wind_speed'].fillna(data['wind_speed'].mean())
data['dew_temperature'] = data['dew_temperature'].fillna(data['dew_temperature'].mean())
data['precip_depth_1_hr'] = data['precip_depth_1_hr'].fillna(data['precip_depth_1_hr'].mean())

In [20]:
del dataset

In [21]:
data.describe()

,building_id,meter,meter_reading,site_id,square_feet,air_temperature,dew_temperature,precip_depth_1_hr,wind_speed,day,hour,week,month,year
count,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,2.021610e+07,20216100.0
mean,7.992780e+02,6.624412e-01,2.117121e+03,7.992232e+00,1.077830e+05,1.598795e+01,7.747429e+00,7.964155e-01,3.377525e+00,1.576123e+01,1.150232e+01,3.006958e+00,6.578453e+00,2016.0
std,4.269133e+02,9.309921e-01,1.532356e+05,5.099060e+00,1.171424e+05,1.092109e+01,1.015343e+01,6.740961e+00,2.257628e+00,8.820535e+00,6.922017e+00,1.997191e+00,3.436662e+00,0.0
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.830000e+02,-2.890000e+01,-3.500000e+01,-1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,2016.0
25%,3.930000e+02,0.000000e+00,1.830000e+01,3.000000e+00,3.252700e+04,8.900000e+00,3.000000e-01,0.000000e+00,2.100000e+00,8.000000e+00,6.000000e+00,1.000000e+00,4.000000e+00,2016.0
50%,8.950000e+02,0.000000e+00,7.877500e+01,9.000000e+00,7.270900e+04,1.670000e+01,8.900000e+00,0.000000e+00,3.100000e+00,1.600000e+01,1.200000e+01,3.000000e+00,7.000000e+00,2016.0
75%,1.179000e+03,1.000000e+00,2.679840e+02,1.300000e+01,1.391130e+05,2.390000e+01,1.600000e+01,0.000000e+00,4.600000e+00,2.300000e+01,1.800000e+01,5.000000e+00,1.000000e+01,2016.0
max,1.448000e+03,3.000000e+00,2.190470e+07,1.500000e+01,8.750000e+05,4.720000e+01,2.610000e+01,3.430000e+02,1.900000e+01,3.100000e+01,2.300000e+01,6.000000e+00,1.200000e+01,2016.0


# Fixing site_id 0 error

In [22]:
data['meter_reading'] = [j*0.2931 if i == 0 else j for i,j in zip(data['site_id'],data['meter_reading'])]
data

,building_id,meter,meter_reading,site_id,primary_use,square_feet,air_temperature,dew_temperature,precip_depth_1_hr,wind_speed,day,hour,week,month,year
0,0,0,0.000,0,Education,7432,25.0,20.0,0.796416,0.0,1,0,4,1,2016
1,1,0,0.000,0,Education,2720,25.0,20.0,0.796416,0.0,1,0,4,1,2016
2,2,0,0.000,0,Education,5376,25.0,20.0,0.796416,0.0,1,0,4,1,2016
3,3,0,0.000,0,Education,23685,25.0,20.0,0.796416,0.0,1,0,4,1,2016
4,4,0,0.000,0,Education,116607,25.0,20.0,0.796416,0.0,1,0,4,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,8.750,15,Entertainment/public assembly,19619,1.7,-5.6,-1.000000,8.8,31,23,5,12,2016
20216096,1445,0,4.825,15,Education,4298,1.7,-5.6,-1.000000,8.8,31,23,5,12,2016
20216097,1446,0,0.000,15,Entertainment/public assembly,11265,1.7,-5.6,-1.000000,8.8,31,23,5,12,2016
20216098,1447,0,159.575,15,Lodging/residential,29775,1.7,-5.6,-1.000000,8.8,31,23,5,12,2016


In [23]:
meter_values = data['meter'].unique()
meter_values

array([0, 3, 1, 2])

# Filtering buildings that use electricity meter

In [24]:
electricMeter = data[data['meter']==0]


# Converting primary use to numiric

In [25]:
from sklearn.preprocessing import LabelEncoder

In [26]:
le = LabelEncoder()
data['primary_use'] = le.fit_transform(data['primary_use'])

In [27]:
data

,building_id,meter,meter_reading,site_id,primary_use,square_feet,air_temperature,dew_temperature,precip_depth_1_hr,wind_speed,day,hour,week,month,year
0,0,0,0.000,0,0,7432,25.0,20.0,0.796416,0.0,1,0,4,1,2016
1,1,0,0.000,0,0,2720,25.0,20.0,0.796416,0.0,1,0,4,1,2016
2,2,0,0.000,0,0,5376,25.0,20.0,0.796416,0.0,1,0,4,1,2016
3,3,0,0.000,0,0,23685,25.0,20.0,0.796416,0.0,1,0,4,1,2016
4,4,0,0.000,0,0,116607,25.0,20.0,0.796416,0.0,1,0,4,1,2016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20216095,1444,0,8.750,15,1,19619,1.7,-5.6,-1.000000,8.8,31,23,5,12,2016
20216096,1445,0,4.825,15,0,4298,1.7,-5.6,-1.000000,8.8,31,23,5,12,2016
20216097,1446,0,0.000,15,1,11265,1.7,-5.6,-1.000000,8.8,31,23,5,12,2016
20216098,1447,0,159.575,15,4,29775,1.7,-5.6,-1.000000,8.8,31,23,5,12,2016


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
data.hist(bins=50, figsize=(20, 15))
plt.savefig("attribute_histogram_plots")
plt.show()